In [ ]:
!pip install numpy scipy matplotlib scikit-learn pandas networkx pgmpy causal-learn


In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from causallearn.search.ConstraintBased.PC import pc

print("Ready.")


In [1]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/nasifnur307-blip/biomarkerlink.git


Mounted at /content/drive
Cloning into 'biomarkerlink'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 25 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 7.19 KiB | 3.59 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
import os

folders = [
    "data/raw",
    "data/processed",
    "results/graphs",
    "results/models",
]

for f in folders:
    os.makedirs(f, exist_ok=True)

print("Folders created.")


Folders created.


In [3]:
file_path = "/content/drive/MyDrive/CHB-MIT/preprocessed_chbmit.csv"
